# Kryptologie LAB - CBC

In [1]:
import numpy as np
import random

## AES

In [2]:
# helper functions https://stackoverflow.com/a/30375198/6600660

def int_to_bytes(x: int) -> bytes:
    return x.to_bytes((x.bit_length() + 7) // 8, 'big')

def int_from_bytes(xbytes: bytes) -> int:
    return int.from_bytes(xbytes, 'big')

In [3]:
encryptGaloisMatrix = np.array([[2,3,1,1],[1,2,3,1],[1,1,2,3],[3,1,1,2]])
decryptGaloisMatrix = np.array([[0xE,0xB,0xD,9],[9,0xE,0xB,0xD],[0xD,9,0xE,0xB],[0xB,0xD,9,0xE]])

with open('SBox.txt') as f:
    lines = f.readlines()
    hexValues = "".join(lines).replace("\n", " ").split(", ")
    byteValues = [int(x, base=16) for x in hexValues]
    sbox = np.reshape(np.array(byteValues, dtype=np.dtype('B')), (16,16))
    print("SBox")
    print(sbox)

with open('SBoxInvers.txt') as f:
    lines = f.readlines()
    hexValues = "".join(lines).replace("\n", " ").split(", ")
    byteValues = [int(x, base=16) for x in hexValues]
    sbox_inverse = np.reshape(np.array(byteValues, dtype=np.dtype('B')), (16,16))
    print("SBox Inverse")
    print(sbox_inverse)

def gfAddition(a,b):
    return np.bitwise_xor(a,b)

def xTimes(a):
    t = a << 1
    if a & (1 << 7) != 0:
        t = t ^ 0x1b
    return t

def gfMultiply(a,b):
    index = 0
    out = 0
    while b != 0:
        if (b & 1 == 1):
            val = a
            for i in range(index):
                val = xTimes(val)
            out = gfAddition(out, val)
        index += 1
        b = b >> 1
    return out

def initBlock(textblock):
    chars = [ord(c) for c in textblock]
    if (len(chars) < 16):
        fill = [0 for i in range(16 - len(chars))]
        chars.extend(fill)
    return np.reshape(np.array(chars, dtype=np.dtype('B')), (4,4),order="F")

def blockToText(textblock):
    chars = [chr(c) for c in np.nditer(textblock, order="F")]
    return "".join(chars)

def addRoundKey(textblock, key):
    return np.bitwise_xor(textblock, key)

def subBytes(textblock, reverse=False):
    if reverse:
        box = sbox_inverse
    else:
        box = sbox
    for y in range(len(textblock)):
        for x in range(len(textblock[y])):
            row = (textblock[y,x] >> 4) & 0xF
            col = textblock[y,x] & 0xF
            textblock[y,x] = box[row, col]
    return textblock

def shiftRows(textblock, reverse=False):
    for i in range(4):
        if reverse:
            shiftAmount = i
        else:
            shiftAmount = -i
        textblock[i] = np.roll(textblock[i], shiftAmount)
    return textblock

def subColumn(inputColumn, matrix):
    outputColumn = np.copy(inputColumn)
    for row in range(4):
        val = 0
        for column in range(4):
            val = gfAddition(val, gfMultiply(matrix[row, column], inputColumn[column]))
        outputColumn[row] = val
    return outputColumn

def mixColumns(textblock, reverse=False):
    if reverse:
        matrix = decryptGaloisMatrix
    else:
        matrix = encryptGaloisMatrix
    for column in range(4):
        textblock[:, column] = subColumn(textblock[:, column], matrix)
    return textblock

def encryptBlock(textblock, keys, inputAsMatrix=False, returnAsBlock=False):
    if not inputAsMatrix:
        textblock = initBlock(textblock)
    textblock = addRoundKey(textblock, keys[0])
    for i in range(1, 10):
        textblock = subBytes(textblock)
        textblock = shiftRows(textblock)
        textblock = mixColumns(textblock)
        textblock = addRoundKey(textblock, keys[i])
    textblock = subBytes(textblock)
    textblock = shiftRows(textblock)
    textblock = addRoundKey(textblock, keys[10])
    if returnAsBlock:
        return textblock
    return blockToText(textblock)

def decryptBlock(textblock, keys):
    textblock = initBlock(textblock)
    textblock = addRoundKey(textblock, keys[10])
    textblock = shiftRows(textblock, reverse=True)
    textblock = subBytes(textblock, reverse=True)
    for i in range(9, 0, -1):
        textblock = addRoundKey(textblock, keys[i])
        textblock = mixColumns(textblock, reverse=True)
        textblock = shiftRows(textblock, reverse=True)
        textblock = subBytes(textblock, reverse=True)
    textblock = addRoundKey(textblock, keys[0])
    return blockToText(textblock)

def encrypt(text, keys):
    blocks = [text[i:i+16] for i in range(0, len(text), 16)]
    return "".join([encryptBlock(block, keys) for block in blocks])

def decrypt(text, keys):
    blocks = [text[i:i+16] for i in range(0, len(text), 16)]
    return "".join([decryptBlock(block, keys) for block in blocks])

SBox
[[ 99 124 119 123 242 107 111 197  48   1 103  43 254 215 171 118]
 [202 130 201 125 250  89  71 240 173 212 162 175 156 164 114 192]
 [183 253 147  38  54  63 247 204  52 165 229 241 113 216  49  21]
 [  4 199  35 195  24 150   5 154   7  18 128 226 235  39 178 117]
 [  9 131  44  26  27 110  90 160  82  59 214 179  41 227  47 132]
 [ 83 209   0 237  32 252 177  91 106 203 190  57  74  76  88 207]
 [208 239 170 251  67  77  51 133  69 249   2 127  80  60 159 168]
 [ 81 163  64 143 146 157  56 245 188 182 218  33  16 255 243 210]
 [205  12  19 236  95 151  68  23 196 167 126  61 100  93  25 115]
 [ 96 129  79 220  34  42 144 136  70 238 184  20 222  94  11 219]
 [224  50  58  10  73   6  36  92 194 211 172  98 145 149 228 121]
 [231 200  55 109 141 213  78 169 108  86 244 234 101 122 174   8]
 [186 120  37  46  28 166 180 198 232 221 116  31  75 189 139 138]
 [112  62 181 102  72   3 246  14  97  53  87 185 134 193  29 158]
 [225 248 152  17 105 217 142 148 155  30 135 233 206  85

In [4]:
def subByte(byte, box):
    row = (byte >> 4) & 0xF
    col = byte & 0xF
    return box[row, col]

def rotWord(word):
    return np.roll(word, shift=-1)

def subWord(word):
    return [subByte(byte, sbox) for byte in word]

def rcon(i):
    array = [0x1, 0x2, 0x4, 0x8, 0x10, 0x20, 0x40, 0x80, 0x1b, 0x36]
    return [array[i], 0,0,0]

# input 16 character string
def generateKey(key):
    block = initBlock(key)
    words = []
    words.append(block[:,0])
    words.append(block[:,1])
    words.append(block[:,2])
    words.append(block[:,3])
    for i in range(4, 44):
        if i % 4 == 0:
            val = gfAddition(words[i-4], rcon(i // 4 - 1))
            val = gfAddition(subWord(rotWord(words[i-1])), val)
            words.append(val)
        else:
            words.append(gfAddition(words[i-4], words[i-1]))
    keys = np.empty((11, 4,4), dtype=np.dtype('B'))
    for i in range(0, 44):
        keys[i // 4, :, i % 4] = words[i]
    return keys

## CBC
![CBC](CBC.PNG)

In [5]:
blockLength = 16
IV = np.zeros((4,4), dtype=np.dtype('B'))
keys = generateKey("9876543210987654")

In [6]:
def getChunks(text, blockLength=16):
    return [text[i:i+blockLength] for i in range(0, len(text), blockLength)]

def CBCMAC(k, chunks):
    xorVec = IV
    output = ""
    for chunk in chunks:
        chunkAsBytes = initBlock(chunk)
        inputBytes = np.bitwise_xor(chunkAsBytes, xorVec)
        xorVec = encryptBlock(inputBytes, k, inputAsMatrix=True, returnAsBlock=True)
        output = xorVec
    return blockToText(output)

In [7]:
text = "Lorem ipsum dolor sit amet, consetetur sadipscing elitr, sed diam nonumy eirmod tempor invidunt ut labore et dolore magna aliquyam erat, sed diam voluptua. At vero eos et accusam et justo duo dolores et ea rebum. Stet clita kasd gubergren, no sea takimata sanctus est Lorem ipsum dolor sit amet. Lorem ipsum dolor sit amet, consetetur sadipscing elitr, sed diam nonumy eirmod tempor invidunt ut labore et dolore magna aliquyam erat, sed diam voluptua. At vero eos et accusam et justo duo dolores et ea rebum. Stet clita kasd gubergren, no sea takimata sanctus est Lorem ipsum dolor sit amet."
chunks = getChunks(text, blockLength)
print("Chunks:\n")
print(chunks)
CBCMAC(keys, chunks)

Chunks:

['Lorem ipsum dolo', 'r sit amet, cons', 'etetur sadipscin', 'g elitr, sed dia', 'm nonumy eirmod ', 'tempor invidunt ', 'ut labore et dol', 'ore magna aliquy', 'am erat, sed dia', 'm voluptua. At v', 'ero eos et accus', 'am et justo duo ', 'dolores et ea re', 'bum. Stet clita ', 'kasd gubergren, ', 'no sea takimata ', 'sanctus est Lore', 'm ipsum dolor si', 't amet. Lorem ip', 'sum dolor sit am', 'et, consetetur s', 'adipscing elitr,', ' sed diam nonumy', ' eirmod tempor i', 'nvidunt ut labor', 'e et dolore magn', 'a aliquyam erat,', ' sed diam volupt', 'ua. At vero eos ', 'et accusam et ju', 'sto duo dolores ', 'et ea rebum. Ste', 't clita kasd gub', 'ergren, no sea t', 'akimata sanctus ', 'est Lorem ipsum ', 'dolor sit amet.']


'§9ëS\x94ú\x9b\x067»Fþ\x94\x87@Ï'

## CCM-Modus (Counter with CBC-MAC)

In [8]:
def initBlockFromInt(number: int):
    bytes = []
    for byte in range(15,-1,-1):
        bytes.append((number >> (8 * byte)) & 0xFF)
    return np.array(bytes, dtype=np.uint8).reshape((4,4)).transpose()

def counter(k, nonce, inputChunks):
    output = ""
    init_ctr = nonce << 64
    # Counter encryption
    for i in range(len(inputChunks)):
        chunkAsBytes = initBlock(inputChunks[i])
        T_i = (init_ctr + i + 1) % (2 ** 128)
        T_i_matrix = initBlockFromInt(T_i)
        encryptedT = encryptBlock(T_i_matrix, k, inputAsMatrix=True, returnAsBlock=True)
        output += blockToText(np.bitwise_xor(chunkAsBytes, encryptedT))
    return output

def encryptAndSignCCM(k, nonce, inputChunks):
    # Counter encryption
    output = counter(k, nonce, inputChunks)
    # CBCMac
    hash = CBCMAC(k, inputChunks)
    output += hash
    return output

def decryptAndVerify(k, nonce, inputChunks):
    message = counter(k, nonce, inputChunks[:-1])
    print("Message:\n", message)
    # CBCMac
    hash = CBCMAC(k, getChunks(message))
    expectedHash =  inputChunks[-1]
    print("exctracted: ",expectedHash)
    assert hash == expectedHash
    return message

In [9]:
nonce = random.getrandbits(64)
print(nonce)

6941386528259508195


In [10]:
# Test
chunks = getChunks(text, blockLength)
output = counter(keys, nonce, chunks)
print(output)
print(counter(keys, nonce, getChunks(output, blockLength)))


á
õèX y/5F=i~/DTK7ú«f½,áõq=]#Ð¯^(Ôôó"Ê¯QÂXzPÆ¿¥$HÍô`ü³m3.s: 'MLg®«å¦þ	©~ 1½~ê¿µfxê +GjÝ®ú*ÔÓÒ{©Ü±¥6%EÀR>ÊV$Y5l\°ÍgAýÝdÅ<¼ËmQº!mõÉúLLÔæäâPüã±1æbËê¦ ¨Ô	ZE)Fîl?¸gPÃArBÒ]ZZûòJ@ZÖMÍ+Cz³7â |ñ ïL'+2xÈÝS4ãÍ'ÃÙùñ¤4q«	¦&E°V\'	öÖTO£ÝaËìîõðé6c¸úç¬VRÝ* êY»ò±{ÜP'ÌÀÍÀ52'ýQÎ?{GM>Î8Ú ÁòS®ð¯Ä«ä^SqÞz¸¯gd¯GvüÎ!A2ÊTø?§NQpÙ¦Úê©q+F_§èÅ? ×ó°»ÊÑõ'rµÇÃó¹ø
Lorem ipsum dolor sit amet, consetetur sadipscing elitr, sed diam nonumy eirmod tempor invidunt ut labore et dolore magna aliquyam erat, sed diam voluptua. At vero eos et accusam et justo duo dolores et ea rebum. Stet clita kasd gubergren, no sea takimata sanctus est Lorem ipsum dolor sit amet. Lorem ipsum dolor sit amet, consetetur sadipscing elitr, sed diam nonumy eirmod tempor invidunt ut labore et dolore magna aliquyam erat, sed diam voluptua. At vero eos et accusam et justo duo dolores et ea rebum. Stet clita kasd gubergren, no sea takimata sanctus est Lorem ipsum dolor

In [11]:
text = "Lorem ipsum dolor sit amet, consetetur sadipscing elitr, sed diam nonumy eirmod tempor invidunt ut labore et dolore magna aliquyam erat, sed diam voluptua. At vero eos et accusam et justo duo dolores et ea rebum. Stet clita kasd gubergren, no sea takimata sanctus est Lorem ipsum dolor sit amet. Lorem ipsum dolor sit amet, consetetur sadipscing elitr, sed diam nonumy eirmod tempor invidunt ut labore et dolore magna aliquyam erat, sed diam voluptua. At vero eos et accusam et justo duo dolores et ea rebum. Stet clita kasd gubergren, no sea takimata sanctus est Lorem ipsum dolor sit amet."
chunks = getChunks(text, blockLength)

y = encryptAndSignCCM(keys, nonce, chunks)
y

'\n²b\x011\x18ÞÃð#|_&Qp\x9c\x13\x97\x11à§\x91FJØ(û\x1d\x991\x11Mg¶Ðm|;\x99\x86Ñ±yÑ\x9c¥ô<Cé\x8fáê\x80x§.vËR\x86\rá\nÛ\x951}\x9a8üÔ\x01À`Gìî·r\x9f¦³\x83Ãn\x8b×`*\x819\x16\x9d~]æ\x88ìRðÓ\x80Úÿ|ýñI\x82g\x11¹\x1e\x8eá\x19¹f`\x8fûhM \x1bf8þúc©²,\x95\x00Ê\x19/QÁê\x96öi\x00<Àâ\x13wàäÁ¢(\x8b-Ç\x00ÇX\x9a\x0f½\\)bé\x8c\rõè\x13X\xa0y/5F=i~\x87\x7f\x86/D\x94TK7\x84\x94\x1e\x93ú\x13«f½\x85,áõ\x90q=\x89]#Ð¯^(Ôôó"\x0bs\x08\x9dÊ\x1e¯QÂXzPÆ¿¥$\x8c\x1eHÍô`ü³m3.s:\xa0\'M\x0fLg®\x95«å¦þ\t©~ 1½~ê¿µ\x9cfxê\xa0+\x1bG\x1b\x11jÝ®ú*\x92ÔÓÒ{\x9b\x07\x8c©Ü±¥6%E\x11ÀR>ÊV\x85$Y5l\\°ÍgAýÝ\x1edÅ<¼ËmQº\x1a!mõ\x8eU\x08É\x94\x9cúLLÔæäâPüã±1æbËÐ\x08ê¦\xa0¨\x93\x97Ô\tZE)\x18Fî\x06l\x02?¸gPÃ\x93Ar\x97\x06B\x8dÒ\x0c]Z\x83Zû\x92òJ@ZÖ\x97\x9eMÍ+Cz\x07³7â\x8b\x00|ñ\xa0\x01ïL\'+2xÈÝS4ãÍ\'ÃÙùñ¤4q\x8e«\x7f\x0f\t¦&E°\x8dV\\\'\töÖTO£\x95\x90\x89Ýa\x02\x06Ë\x86ìîõ\x88\x9bð\x88é6c¸úç¬VRÝ*\xa0êY»ò\x97±{ÜP\'\x8cÌÀÍÀ5\x042\'ýQ\x8cÎ\x8a?{\x1eGM>Î8Ú Á\x89ò\x80S®ð¯Ä«ä\x16^Sq\x8bÞz¸¯gd¯G\x98vüÎ!A\x932Ê\x9fTø?§\x84\x97NQpÙ¦Úê©q+F_§èÅ?\x00×ó\

In [12]:
message = decryptAndVerify(keys, nonce, getChunks(y))

Message:
 Lorem ipsum dolor sit amet, consetetur sadipscing elitr, sed diam nonumy eirmod tempor invidunt ut labore et dolore magna aliquyam erat, sed diam voluptua. At vero eos et accusam et justo duo dolores et ea rebum. Stet clita kasd gubergren, no sea takimata sanctus est Lorem ipsum dolor sit amet. Lorem ipsum dolor sit amet, consetetur sadipscing elitr, sed diam nonumy eirmod tempor invidunt ut labore et dolore magna aliquyam erat, sed diam voluptua. At vero eos et accusam et justo duo dolores et ea rebum. Stet clita kasd gubergren, no sea takimata sanctus est Lorem ipsum dolor sit amet. 
exctracted:  §9ëSú7»Fþ@Ï
